In [34]:
# import the libraries as shown below

from tensorflow.keras.layers import Input, Lambda, Dense, Flatten 
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet50 import ResNet50
from keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [18]:
# re-size all the images to this
IMAGE_SIZE = [224, 224]#vgg16 input size is 224,224

train_path = 'dataset/training_set'
valid_path = 'dataset/test_set'

In [19]:
# Import the Vgg 16 library as shown below and add preprocessing layer to the front of VGG
# Here we will be using imagenet weights

vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)




In [20]:
# don't train existing weights
#very imp as weights are fixed ...it is a state of art algo

for layer in vgg.layers:
    layer.trainable = False

In [21]:
  # useful for getting number of output classes
    #how many folders inside train dataset describes no.of classes
folders = glob('dataset/training_set/*')
folders

['dataset/training_set\\anand', 'dataset/training_set\\not_anand']

In [22]:
# our layers - you can add more if you want
x = Flatten()(vgg.output)
#

In [29]:
prediction = Dense(1, activation='sigmoid')(x)
#we cut the last layer of resnet and added 3 categ
# create a model object
model = Model(inputs=vgg.input, outputs=prediction)

In [30]:
model.summary()


Model: "functional_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)      

In [31]:
# tell the model what cost and optimization method to use
model.compile(
  loss='binary_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)


In [35]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

In [36]:
training_set = train_datagen.flow_from_directory('dataset/training_set',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

Found 38 images belonging to 2 classes.


In [34]:
test_datagen = ImageDataGenerator(rescale = 1./255)

test_set = test_datagen.flow_from_directory('dataset/test_set',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 38 images belonging to 2 classes.


In [35]:
# fit the model
# Run the cell. It will take some time to execute
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=50,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

Epoch 1/50
2/2 [==============================] - 33s 17s/step - loss: 0.9672 - accuracy: 0.4474 - val_loss: 0.7624 - val_accuracy: 0.5000
Epoch 2/50
2/2 [==============================] - 15s 8s/step - loss: 0.5588 - accuracy: 0.6053 - val_loss: 0.2219 - val_accuracy: 0.9737
Epoch 3/50
2/2 [==============================] - 27s 14s/step - loss: 0.4411 - accuracy: 0.7368 - val_loss: 0.3895 - val_accuracy: 0.7895
Epoch 4/50
2/2 [==============================] - 28s 14s/step - loss: 0.3580 - accuracy: 0.8158 - val_loss: 0.1151 - val_accuracy: 0.9737
Epoch 5/50
2/2 [==============================] - 30s 15s/step - loss: 0.0825 - accuracy: 1.0000 - val_loss: 0.0441 - val_accuracy: 1.0000
Epoch 6/50
2/2 [==============================] - 20s 10s/step - loss: 0.0826 - accuracy: 0.9737 - val_loss: 0.0666 - val_accuracy: 1.0000
Epoch 7/50
2/2 [==============================] - 30s 15s/step - loss: 0.0798 - accuracy: 1.0000 - val_loss: 0.0872 - val_accuracy: 1.0000
Epoch 8/50
2/2 [============

In [36]:

from tensorflow.keras.models import load_model

model.save('anand_vgg16_model.h5')

In [2]:
from tensorflow.keras.models import load_model
cnn= load_model('anand_vgg16_model.h5')

In [37]:
training_set.class_indices#print this to know whether 0=cat or dog 

{'anand': 0, 'not_anand': 1}

In [39]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('dataset/single_prediction/3.jpg', target_size = (224, 224))
print(test_image)
test_image = image.img_to_array(test_image)#predict method expects a 2 d array
test_image = np.expand_dims(test_image, axis = 0)#to fit in batch of 32
result = cnn.predict(test_image)
print(result)
#training_set.class_indices#print this to know whether 0=cat or dog 
if result[0][0] == 0:
  prediction = 'anand'
else:
  prediction = 'unknown'

<PIL.Image.Image image mode=RGB size=224x224 at 0x286828C1F10>
[[2.307903e-20]]


In [40]:
print(prediction)

unknown
